In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

In [ ]:
!conda install paddlepaddle-gpu==2.4.1 cudatoolkit=11.2 -c https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/Paddle/ -c conda-forge

In [ ]:
!git clone https://github.com/yeyupiaoling/PPASR.git
%cd PPASR
!python setup.py install

In [ ]:
!python -m pip install ppasr -U

In [22]:
!python -m pip install paddlespeech-ctcdecoders -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.8 MB 7.3 MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip /content/drive/MyDrive/PPASR_V2-conformer_online-fbank-超大数据集.zip -d /content

Archive:  /content/drive/MyDrive/PPASR_V2-conformer_online-fbank-超大数据集.zip
   creating: /content/PPASR_V2-conformer_online-fbank-超大数据集/
   creating: /content/PPASR_V2-conformer_online-fbank-超大数据集/configs/
  inflating: /content/PPASR_V2-conformer_online-fbank-超大数据集/configs/conformer_online_zh.yml  
   creating: /content/PPASR_V2-conformer_online-fbank-超大数据集/dataset/
  inflating: /content/PPASR_V2-conformer_online-fbank-超大数据集/dataset/mean_istd.json  
  inflating: /content/PPASR_V2-conformer_online-fbank-超大数据集/dataset/vocabulary.txt  
   creating: /content/PPASR_V2-conformer_online-fbank-超大数据集/models/
   creating: /content/PPASR_V2-conformer_online-fbank-超大数据集/models/conformer_online_fbank/
   creating: /content/PPASR_V2-conformer_online-fbank-超大数据集/models/conformer_online_fbank/best_model/
  inflating: /content/PPASR_V2-conformer_online-fbank-超大数据集/models/conformer_online_fbank/best_model/model.pdparams  


In [9]:
!cp -RT /content/PPASR_V2-conformer_online-fbank-超大数据集/configs/ /content/PPASR/configs/
!cp -RT /content/PPASR_V2-conformer_online-fbank-超大数据集/dataset/ /content/PPASR/dataset/
!cp -RT /content/PPASR_V2-conformer_online-fbank-超大数据集/models/ /content/PPASR/models/

In [ ]:
!python export_model.py --resume_model=models/conformer_online_fbank/best_model/

In [95]:
!unzip /content/drive/MyDrive/data.zip -d /content/PPASR/raw_wav

Archive:  /content/drive/MyDrive/data.zip
  inflating: /content/PPASR/raw_wav/lj1216.wav  
  inflating: /content/PPASR/raw_wav/aam0511.wav  
  inflating: /content/PPASR/raw_wav/alv0805.wav  
  inflating: /content/PPASR/raw_wav/hp0317.wav  


In [89]:
!mkdir /content/PPASR/slcied_wav

In [99]:
%cd /content/PPASR
import glob
import os
import shutil
import wave

import librosa
import soundfile

def length(src: str):
    if os.path.isfile(src) and src.endswith('.wav'):
        with wave.open(src, 'r') as w:
            return w.getnframes() / w.getframerate() / 3600
    elif os.path.isdir(src):
        total = 0
        for ch in [os.path.join(src, c) for c in os.listdir(src)]:
            total += length(ch)
        return total
    return 0


print('Environment initialized successfully.')

# Configuration for data paths
raw_path = '/content/PPASR/raw_wav'  # Path to your raw, unsliced recordings

########################################

assert os.path.exists(raw_path) and os.path.isdir(raw_path), 'The chosen path does not exist or is not a directory.'
print('Raw recording path:', raw_path)
print()
print('===== Recording List =====')
raw_filelist = glob.glob(f'{raw_path}/*.wav', recursive=True)
raw_length = length(raw_path)
if len(raw_filelist) > 5:
    print('\n'.join(raw_filelist[:5] + [f'... ({len(raw_filelist) - 5} more)']))
else:
    print('\n'.join(raw_filelist))
print()
print(f'Found {len(raw_filelist)} valid recordings with total length of {round(raw_length, 2)} hours.')

sliced_path = '/content/PPASR/sliced_wav'  # Path to hold the sliced segments of your recordings

# Slicer arguments
db_threshold_ = -40.
min_length_ = 15000
win_l_ = 800
win_s_ = 40
max_silence_kept_ = 1000

# Number of threads (based on your CPU kernels)
num_workers = 5

########################################

assert 'raw_path' in locals().keys(), 'Raw path of your recordings has not been specified.'
assert not os.path.exists(sliced_path) or os.path.isdir(sliced_path), 'The chosen path is not a directory.'
os.makedirs(sliced_path, exist_ok=True)
print('Sliced recording path:', sliced_path)

from slicer import Slicer
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED

def slice_one(in_audio):
    audio, sr = librosa.load(in_audio, sr=None)
    slicer = Slicer(
        sr=sr,
        db_threshold=db_threshold_,
        min_length=min_length_,
        win_l=win_l_,
        win_s=win_s_,
        max_silence_kept=max_silence_kept_
    )
    chunks = slicer.slice(audio)
    for i, chunk in enumerate(chunks):
        soundfile.write(os.path.join(sliced_path, f'%s_slice_%04d.wav' % (os.path.basename(in_audio).rsplit('.', maxsplit=1)[0], i)), chunk, sr)


print('Slicing your recordings may take several minutes. Please wait.')
thread_pool = ThreadPoolExecutor(max_workers=num_workers)
tasks = []
for file in raw_filelist:
    tasks.append(thread_pool.submit(slice_one, file))
wait(tasks, return_when=ALL_COMPLETED)
print()
print('===== Segment List =====')
sliced_filelist = glob.glob(f'{sliced_path}/*.wav', recursive=True)
sliced_length = length(sliced_path)
if len(sliced_filelist) > 5:
    print('\n'.join(sliced_filelist[:5] + [f'... ({len(sliced_filelist) - 5} more)']))
else:
    print('\n'.join(sliced_filelist))
print()
print(f'Sliced your recordings into {len(sliced_filelist)} segments with total length of {round(sliced_length, 2)} hours.')

/content/PPASR
Environment initialized successfully.
Raw recording path: /content/PPASR/raw_wav

===== Recording List =====
/content/PPASR/raw_wav/lj1216.wav
/content/PPASR/raw_wav/alv0805.wav
/content/PPASR/raw_wav/hp0317.wav
/content/PPASR/raw_wav/aam0511.wav

Found 4 valid recordings with total length of 0.37 hours.
Sliced recording path: /content/PPASR/sliced_wav
Slicing your recordings may take several minutes. Please wait.
executing 'slice' costed 27.863s
executing 'slice' costed 28.548s
executing 'slice' costed 35.624s
executing 'slice' costed 38.519s

===== Segment List =====
/content/PPASR/sliced_wav/hp0317_slice_0008.wav
/content/PPASR/sliced_wav/aam0511_slice_0018.wav
/content/PPASR/sliced_wav/aam0511_slice_0013.wav
/content/PPASR/sliced_wav/lj1216_slice_0011.wav
/content/PPASR/sliced_wav/aam0511_slice_0009.wav
... (55 more)

Sliced your recordings into 60 segments with total length of 0.31 hours.


In [ ]:
%%shell
cd /content/PPASR
ls ./sliced_wav/*.wav


for file in ./sliced_wav/*.wav;  #一级目录下的内容-->并不递归显示！ 
  do  
     python infer_path.py --wav_path=$file;  #${file}代表的是文件的全路径
  done 

./sliced_wav/aam0511_slice_0000.wav  ./sliced_wav/alv0805_slice_0007.wav
./sliced_wav/aam0511_slice_0001.wav  ./sliced_wav/alv0805_slice_0008.wav
./sliced_wav/aam0511_slice_0002.wav  ./sliced_wav/alv0805_slice_0009.wav
./sliced_wav/aam0511_slice_0003.wav  ./sliced_wav/alv0805_slice_0010.wav
./sliced_wav/aam0511_slice_0004.wav  ./sliced_wav/alv0805_slice_0011.wav
./sliced_wav/aam0511_slice_0005.wav  ./sliced_wav/alv0805_slice_0012.wav
./sliced_wav/aam0511_slice_0006.wav  ./sliced_wav/alv0805_slice_0013.wav
./sliced_wav/aam0511_slice_0007.wav  ./sliced_wav/alv0805_slice_0014.wav
./sliced_wav/aam0511_slice_0008.wav  ./sliced_wav/hp0317_slice_0000.wav
./sliced_wav/aam0511_slice_0009.wav  ./sliced_wav/hp0317_slice_0001.wav
./sliced_wav/aam0511_slice_0010.wav  ./sliced_wav/hp0317_slice_0002.wav
./sliced_wav/aam0511_slice_0011.wav  ./sliced_wav/hp0317_slice_0003.wav
./sliced_wav/aam0511_slice_0012.wav  ./sliced_wav/hp0317_slice_0004.wav
./sliced_wav/aam0511_slice_0013.wav  ./sliced_wav/hp0317